In [ ]:
import pandas as pd
import requests 
import cbsodata
import re
# from sklearn.linear_model import LogisticRegression


## Opmerkingen
Het is nu niet mogelijk m helder te krijgen wat de kolommen in een tabel precies betekenen, daarvoor is de oude ingestion methode nodig volgens mij

## New ingestion method

In [ ]:
import pandas as pd

def split_period_column(df, perioden_column_name='Perioden', year_column_name='Jaar', period_column_name='Periode', quarter_column_name='Kwartaal'):
    """
    Splits the 'Perioden' column of a DataFrame into 'Jaar' (year), 'Periode' (remaining part), and 'Kwartaal' (quarter).

    Parameters:
        df (DataFrame): The DataFrame containing the 'Perioden' column.
        perioden_column_name (str): The name of the column containing the period data. Default is 'Perioden'.
        year_column_name (str): The name of the column to store the extracted years. Default is 'Jaar'.
        period_column_name (str): The name of the column to store the remaining part of the period. Default is 'Periode'.
        quarter_column_name (str): The name of the column to store the extracted quarter. Default is 'Kwartaal'.

    Returns:
        DataFrame: The DataFrame with the 'Perioden' column split into 'Jaar', 'Periode', and 'Kwartaal'.
    """
    # Copy the original DataFrame to avoid modifying the original
    df_copy = df.copy()

    # Split the 'Perioden' column
    df_copy[[year_column_name, period_column_name]] = df_copy[perioden_column_name].str.extract(r'(\d{4})\s?(.*)')

    # Extract the quarter information
    df_copy[quarter_column_name] = df_copy[period_column_name].apply(lambda x: extract_quarter(x))

    return df_copy

def extract_quarter(period_part):
    """
    Extracts the quarter information from the remaining part of the period.

    Parameters:
        period_part (str): The remaining part of the period.

    Returns:
        int: The quarter number if found, otherwise None.
    """
    for char in period_part:
        if char.isdigit():
            return int(char)
    return None

# Example usage:
# ziekteverzuim_data = pd.DataFrame(cbsodata.get_data('80072ned'))
# ziekteverzuim_data = split_period_column(ziekteverzuim_data)
# print(ziekteverzuim_data.head())


In [ ]:
ziekteverzuim_data = pd.DataFrame(cbsodata.get_data('80072ned'))
ziekteverzuim_data = split_period_column(ziekteverzuim_data)
print(ziekteverzuim_data.head())

In [ ]:
inflatie_data = pd.DataFrame(cbsodata.get_data('83131NED'))
inflatie_data = split_period_column(inflatie_data)
print(inflatie_data.head())

In [ ]:
# Define the desired frequencies
desired_frequencies = [
    "Fourtimesayear", "Monthly", "Threemonthly", "Quarterly", "Viermaalperjaar",
    "Perkwartaal", "Perweek", "Stopgezet", "Permaand", "Discontinued",
    "Perdriemaanden", "Weekly"
]

# # Define the desired frequencies
# desired_frequencies = [
#     "Fourtimesayear", "Threemonthly", "Quarterly", "Viermaalperjaar",
#     "Perkwartaal", "Perdriemaanden"
# ]

# Fetch the list of tables
tables = cbsodata.get_table_list()

# Function to extract the first and last four-digit numbers from a period string
def extract_years(period):
    # Find all sequences of four digits
    years = re.findall(r'\b\d{4}\b', period)
    if len(years) >= 2:
        return int(years[0]), int(years[-1])
    elif len(years) == 1:
        return int(years[0]), int(years[0])
    else:
        return None, None

# List to store the identifiers, short titles, and languages
filtered_data = []

# Filter tables and extract identifiers, short titles, and languages
for table in tables:
    if table.get('Frequency') in desired_frequencies:
        period = table.get('Period', 'N/A')
        start_year, end_year = extract_years(period)
        if start_year is not None and end_year is not None:
            if start_year <= 2012 and end_year >= 2022:
                short_title = table.get('ShortTitle', 'N/A')
                # Filter out rows containing 'Caribisch' or 'Bonaire' in ShortTitle
                if 'Caribisch' not in short_title and 'Bonaire' not in short_title:
                    identifier = table.get('Identifier', 'N/A')
                    language = table.get('Language', 'N/A')
                    if language == 'nl':  # Only include rows where Language is 'nl'
                        filtered_data.append((identifier, short_title, language))

# Print the filtered data
print("Filtered identifiers, short titles, and languages:")
for identifier, short_title, language in filtered_data:
    print(f"Identifier: {identifier}, ShortTitle: {short_title}, Language: {language}")


In [ ]:
# Fetch the list of tables
tables = cbsodata.get_table_list()

# Initialize sets for unique frequencies and unique periods
unique_frequencies = set()
unique_periods = set()
unique_languages = set()

# Loop through the tables and collect unique frequencies and periods
for table in tables:
    frequency = table.get('Frequency', 'N/A')
    period = table.get('Period', 'N/A')
    language = table.get('Language','N/A')
    unique_frequencies.add(frequency)
    unique_periods.add(period)
    unique_languages.add(language)

# Convert sets to lists for easier manipulation (optional)
unique_frequencies = list(unique_frequencies)
unique_periods = list(unique_periods)
unique_languages = list(unique_languages)

# # Print unique frequencies
# print("Unique frequencies:")
# for frequency in unique_frequencies:
#     print(frequency)

# # Print unique periods
# print("\nUnique periods:")
# for period in unique_periods:
#     print(period)

# Print unique periods
print("\nUnique languages:")
for language in unique_languages:
    print(language)    

## Old ingestion method

In [ ]:
# Define the API endpoint
api_url = "https://opendata.cbs.nl/ODataApi/odata/80072ned/TypedDataSet"

# Function to get data from the API with pagination handling
def get_data(url):
    data = []
    while url:
        response = requests.get(url)
        if response.status_code == 200:
            json_response = response.json()
            data.extend(json_response['value'])
            url = json_response.get('@odata.nextLink', None)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break
    return data

# Fetch the dataset
data = get_data(api_url)

# Convert the data to a pandas DataFrame
if data:
    df = pd.DataFrame(data)
    print("Data fetched and converted to DataFrame successfully")
else:
    print("Failed to fetch or convert data")

# Save the DataFrame to a CSV file
df.to_csv('sick_leave_data.csv', index=False)
print("Data saved to sick_leave_data.csv")

In [ ]:
# Explore the DataFrame
# View the first few rows
print(df.head())

# View the last few rows
print(df.tail())

# Get summary information
print(df.info())

# Get a statistical summary
print(df.describe())

# View column names
print(df.columns)

# Check for missing values
print(df.isnull().sum())

# Check unique object types
print(df.select_dtypes(include='object').nunique())
